In [0]:
df = spark.read.format("parquet").load("abfss://bronze@eklavyadatabricks.dfs.core.windows.net/orders")

In [0]:
from pyspark.sql.functions import to_timestamp,col,year,dense_rank,rank,row_number
from pyspark.sql.window import Window

df = df.withColumnRenamed("_rescued_data","rescued_data")
df = df.drop("rescued_data")
df = df.withColumn("order_date",to_timestamp(col("order_date")))
df = df.withColumn("year",year(col("order_date")))


rnk = Window.partitionBy("year").orderBy(col("total_amount").desc())
df1 = df.withColumn("flag",dense_rank().over(rnk))
df1 = df1.withColumn("rank_flag",rank().over(rnk))
df1 = df1.withColumn("row_flag",row_number().over(rnk))

display(df1)

## CLASSES - OOP


In [0]:
class Windows:
    rnk = Window.partitionBy("year").orderBy(col("total_amount").desc())
    def dense_rank(self,df):
        return df.withColumn("flag",dense_rank().over(rnk))
    def rank(self,df):
        return df.withColumn("rank_flag",rank().over(rnk))
    def row_number(self,df):
        return df.withColumn("row_flag",row_number().over(rnk))


In [0]:
df_test = df
w = Windows()
df_result=w.dense_rank(df_test)
display(df_result)

In [0]:
display(df)

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@eklavyadatabricks.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.orders
USING DELTA
LOCATION 'abfss://silver@eklavyadatabricks.dfs.core.windows.net/orders'